In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.0.1+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 98.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 42.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch
import numpy as np
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid

# GCN

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, output_dim):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, output_dim)

    def forward(self, x, edge_index, mode=1):
        if mode == 1:
            x = F.dropout(x,p = 0.5 ,training=self.training)
            x = self.conv1(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x,p = 0.5 ,training=self.training)
            x = self.conv2(x, edge_index)
            x = F.dropout(x,p = 0.05 ,training=self.training)
            x = F.relu(x)
            return F.log_softmax(x, dim=1)
        elif mode == 2:
            x = F.dropout(x,p = 0.5 ,training=self.training)
            x = self.conv1(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x,p = 0.5 ,training=self.training)
            x = self.conv2(x, edge_index)
            x = F.dropout(x,p = 0.05 ,training=self.training)
            x = F.relu(x)
            return x

# GAT

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GATConv, Linear
from torch_geometric.datasets import Planetoid


# Define the GAT model
class GAT(torch.nn.Module):
    def __init__(self, num_features, output_dim, embedding_dimension=32, num_heads=8):
        super(GAT, self).__init__()
        self.conv1 = GATConv(num_features, embedding_dimension, heads=num_heads, dropout=0.6)
        self.conv2 = GATConv(num_heads * embedding_dimension, output_dim, heads=num_heads, dropout=0.6)
        self.linear1 = Linear(embedding_dimension * 2 * num_heads, 32)
        # self.linear2 = Linear(32, 16)
        # self.linear3 = Linear(16, 8)

    def forward(self, x, edge_index, mode=1):
        if mode == 1:
            # Pass the input through the first GAT layer
            x = F.dropout(x, p = 0.6 , training=self.training)
            x = self.conv1(x, edge_index)
            x = F.elu(x)
            x = F.dropout(x, p = 0.6 , training=self.training)
            x = self.conv2(x, edge_index)
            return F.log_softmax(x, dim=1)
        elif mode == 2:
            # Pass the input through the first GAT layer
            x = F.dropout(x, p = 0.6 , training=self.training)
            x = self.conv1(x, edge_index)
            x = F.elu(x)
            x = F.dropout(x, p = 0.6 , training=self.training)
            x = self.conv2(x, edge_index)

            return x


# GATv2

In [ ]:

import torch
import torch.nn.functional as F
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv, Linear
from torch_geometric.datasets import Planetoid


# Define the GAT model
class GATv2(torch.nn.Module):
    def __init__(self, num_features, output_dim, embedding_dimension=16, num_heads=8):
        super(GATv2, self).__init__()
        self.conv1 = GATv2Conv(num_features, embedding_dimension, heads=num_heads, dropout=0.6)
        self.conv2 = GATv2Conv(num_heads * embedding_dimension, embedding_dimension, heads=num_heads, dropout=0.6)
        self.linear1 = Linear(embedding_dimension * num_heads, output_dim)
        self.linear2 = Linear(embedding_dimension, output_dim)

    def forward(self, x, edge_index, mode=1):
        if mode == 1:
            # Pass the input through the first GAT layer
            x = F.dropout(x, p = 0.6 , training=self.training)
            x = self.conv1(x, edge_index)
            x = F.elu(x)
            x = F.dropout(x, p = 0.6 , training=self.training)
            x = self.conv2(x, edge_index)
            return F.log_softmax(x, dim=1)
        elif mode == 2:
            # Pass the input through the first GAT layer
            x = F.dropout(x, p = 0.6 , training=self.training)
            x = self.conv1(x, edge_index)
            x = F.elu(x)
            x = F.dropout(x, p = 0.6 , training=self.training)
            x = self.conv2(x, edge_index)
            return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
device

device(type='cuda')

In [ ]:
# device = torch.device('cpu')
device

device(type='cuda')

In [ ]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]


Processing...
Done!


# Create new masks

In [ ]:
import random


def make_random_mask(size, train_fraction, validation_fraction=0):
    new_order = random.sample(range(size), size)

    ls_train = np.zeros(size, dtype=bool)
    ls_validation = np.zeros(size, dtype=bool)
    ls_test = np.zeros(size, dtype=bool)

    for i, value in enumerate(new_order):
        if i < int(size * train_fraction):
            ls_train[value] = True
            ls_validation[value] = False
            ls_test[value] = False
        elif i < int(size * (train_fraction + validation_fraction)):
            ls_train[value] = False
            ls_validation[value] = True
            ls_test[value] = False
        else:
            ls_train[value] = False
            ls_validation[value] = False
            ls_test[value] = True

    ls_train = torch.from_numpy(ls_train)
    ls_validation = torch.from_numpy(ls_validation)
    ls_test = torch.from_numpy(ls_test)

    return ls_train, ls_validation, ls_test


In [ ]:
def train_GNN_mode_and_save(model, data, epochs, train_mask, test_mask, gnn_name="GNN_embeds.pt"):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)

    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = F.nll_loss(out[train_mask], data.y[train_mask])
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 50 == 0:
            model.eval()
            pred = model(data.x, data.edge_index).argmax(dim=1)
            correct = (pred[test_mask] == data.y[test_mask]).sum()
            acc = int(correct) / int(test_mask.sum())
            print(f'epoch : {epoch + 1} = >  Accuracy: {acc:.4f}')

    model.eval()
    pred = model(data.x, data.edge_index).argmax(dim=1)
    correct = (pred[test_mask] == data.y[test_mask]).sum()
    acc = int(correct) / int(test_mask.sum())
    print(f'Final accuracy: {acc:.4f}')

    # now save
    model.eval()
    pred_emb = model(data.x, data.edge_index, mode=2)
    torch.save(pred_emb, gnn_name)

    return acc, pred[test_mask], data.y[test_mask]


# Ecoder decodre

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader
import math

# Prepare data
features = data.x  # Feature matrix
num_features = features.shape[1]

In [ ]:
import torch.nn.functional as F


# Define autoencoder
class SymmetricAutoencoder(nn.Module):
    def __init__(self, input_size, embedding_size):
        super(SymmetricAutoencoder, self).__init__()

        self.encoder_layers = []
        self.decoder_layers = []

        # Encoder
        size = input_size
        while size > embedding_size:
            next_size = 2 ** int(math.log2(size) - 1)
            self.encoder_layers.append(nn.Linear(size, next_size))
            self.encoder_layers.append(nn.ReLU())
            size = next_size

        self.embedding = nn.Linear(size, embedding_size)

        # Decoder

        size = embedding_size

        while size < input_size:
            self.decoder_layers.append(nn.Linear(size, size * 2))
            self.encoder_layers.append(nn.ReLU())
            size = size * 2

        self.decoder_layers.append(nn.Linear(size, input_size))

        self.encoder = nn.Sequential(*self.encoder_layers)
        self.decoder = nn.Sequential(*self.decoder_layers)

    def forward(self, x):
        encoded = self.encoder(x)
        embedded = self.embedding(encoded)
        decoded = self.decoder(embedded)
        return decoded



In [ ]:
def make_enc_embeds(data_features, num_features, num_epochs=5000, embedding_size=32, file_name="encoder_32.pt"):
    # Initialize model and optimizer
    model = SymmetricAutoencoder(num_features, embedding_size).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.00003)
    criterion = nn.MSELoss()

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        reconstructed = model(data_features)
        loss = criterion(reconstructed, data_features)
        loss.backward()
        optimizer.step()
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}")

    # Now you can use the encoder part of the trained autoencoder to get embeddings
    encoded_features = model.encoder(data_features)
    torch.save(encoded_features, file_name)

In [ ]:
import random
def split_data(dataset, num_each_class_train, num_val=500, num_test=1000):
    data = dataset[0]
    y = torch.detach(data.y).numpy()
    keys = [x for x in range(len(y))]
    y_dictionary = dict(zip(keys, y))
    score_list = set(y)
    dic_list = [[] for i in range(len(score_list))]

    for item in y_dictionary:
        dic_list[y_dictionary[item]].append(item)

    l = []
    for sample_list in (dic_list):
        s_list = random.sample(sample_list, num_each_class_train)
        l.extend(s_list)

    train_mask = [False for i in range(len(data.x))]
    for num in l:
        train_mask[num] = True
    mylist = [x for x in range(len(data.x))]
    mylist = [elt for elt in mylist if elt not in l]
    l1 = random.sample(mylist, num_val)
    mylist = [elt for elt in mylist if elt not in l1]
    val_mask = [False for i in range(len(data.x))]
    for num in l1:
        val_mask[num] = True

    l2 = random.sample(mylist, num_test)
    test_mask = [False for i in range(len(data.x))]
    for num in l2:
        test_mask[num] = True

    return train_mask, val_mask, test_mask

# load ,concat  and finalize

In [ ]:
class DNN(torch.nn.Module):
    def __init__(self, input_size, num_classes, hidden_layer=64):
        super().__init__()
        self.linear1 = Linear(input_size, hidden_layer)
        self.linear2 = Linear(hidden_layer, hidden_layer)
        self.linear3 = Linear(hidden_layer, num_classes)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.dropout(x, p=0.2)
        x = F.relu(self.linear2(x))
        x = F.dropout(x, p=0.2)
        x = F.relu(self.linear3(x))
        return F.log_softmax(x, dim=1)

In [ ]:
def finalizer_DNN(train_mask, test_mask, hidden_layer_DNN_size=256, repeat_threshold=0.85,lr=0.001, epochs=3000,
                  file_name_gnn_emb="GNN_embeds.pt",
                  file_name_enc_emb="encoder_emb32_cora.pt", validation_mask = None , early_stop=0.9):
    gnn_emb = torch.load(file_name_gnn_emb).to(device)
    enc_emb = torch.load(file_name_enc_emb).to(device)

    concated_data = torch.cat((gnn_emb, enc_emb), dim=1)

    input_size = concated_data.shape[1]
    model = DNN(input_size, dataset.num_classes, hidden_layer=hidden_layer_DNN_size).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(concated_data)
        loss = F.nll_loss(out[train_mask], data.y[train_mask])
        loss.backward()
        optimizer.step()

        if epoch % 50 == 0:
            model.eval()

            pred = model(concated_data).argmax(dim=1)
            correct = (pred[train_mask] == data.y[train_mask]).sum()
            acc_train = int(correct) / int(train_mask.sum())

            correct = (pred[test_mask] == data.y[test_mask]).sum()
            acc = int(correct) / int(test_mask.sum())
            print(f'epoch {epoch} ,Accuracy: {acc:.4f} , train acc : {acc_train}')

            if validation_mask is not None:
              correct = (pred[validation_mask] == data.y[validation_mask]).sum()
              acc_validation = int(correct) / int(validation_mask.sum())
              if  acc_validation > early_stop:
                break


    # calculate train accuracy
    pred = model(concated_data).argmax(dim=1)
    correct = (pred[new_train_mask] == data.y[new_train_mask]).sum()
    acc_train = int(correct) / int(new_train_mask.sum())

    if (acc_train < repeat_threshold):
        return finalizer_DNN(train_mask, test_mask, repeat_threshold=repeat_threshold, epochs=epochs,
                             file_name_gnn_emb=file_name_gnn_emb,
                             file_name_enc_emb=file_name_enc_emb)

    model.eval()
    pred = model(concated_data).argmax(dim=1)
    correct = (pred[test_mask] == data.y[test_mask]).sum()
    acc = int(correct) / int(test_mask.sum())
    print(f'Accuracy: {acc:.4f}')
    return acc, pred[test_mask], data.y[test_mask]


# ALL scenarios

In [ ]:
def print_details(acc_list, f1_score_list):
    print(f'Accuracy avg = {statistics.mean(acc_list)}')
    print(f'Accuracy deviation = {statistics.stdev(acc_list)}')
    print(f'F1 score(macro) avg = {statistics.mean(f1_score_list)}')
    print(f'F1 score(macro) deviation = {statistics.stdev(f1_score_list)}')

In [ ]:
import statistics
from sklearn.metrics import f1_score
import copy

def run_model(GNN_model , Gnn_epochs ,enc_address ,reply_threshold , head_epochs, validation_mask =None , early_stop = 0.9 ,hidden_layer_DNN_size= 256):
  f1_score_list_base = []
  acc_list_base = []

  f1_score_list = []
  acc_list = []
  for i in range(10):

      model = copy.deepcopy(GNN_model)
      print("*****")
      print(f'epoch : {i + 1} / 10')
      # model = GATv2(dataset.num_features, dataset.num_classes, embedding_dimension=32).to(device)
      # GNN_model = GAT(dataset.num_features, dataset.num_classes, embedding_dimension=128).to(device)
      # model = GCN(128, dataset.num_classes).to(device)
      acc_base, preds_base, labels_base = train_GNN_mode_and_save(model, data, Gnn_epochs, train_mask=new_train_mask,
                                                                  test_mask=new_test_mask)
      preds_base = preds_base.cpu().numpy()
      labels_base = labels_base.cpu().numpy()
      f1_score_base = f1_score(labels_base, preds_base, average='macro')

      f1_score_list_base.append(f1_score_base)
      acc_list_base.append(acc_base)

      acc, preds, labels = finalizer_DNN(train_mask=new_train_mask, epochs=head_epochs, test_mask=new_test_mask,
                                        repeat_threshold=reply_threshold,
                                        file_name_enc_emb=enc_address,
                                        validation_mask = validation_mask,
                                        early_stop = early_stop
                                        hidden_layer_DNN_size = hidden_layer_DNN_size)

      preds = preds.cpu().numpy()
      labels = labels.cpu().numpy()

      print("base model")
      print(acc_base)
      print(f1_score_base)

      print("OurModel")
      print(acc)
      print(f1_score(labels, preds, average='macro'))
      f1_score_list.append(f1_score(labels, preds, average='macro'))
      acc_list.append(acc)


  print("+++***Fianal Result***+++")
  print("base")
  print_details(acc_list_base, f1_score_list_base)
  print("new model")
  print_details(acc_list, f1_score_list)

# Cora

In [ ]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

# Prepare data
features = data.x  # Feature matrix
num_features = features.shape[1]

new_train_mask, new_valdation_mask, new_test_mask = make_random_mask(data.x.shape[0], 0.5, 0.2)
data = data.to(device)

In [ ]:
make_enc_embeds(data.x, num_features, num_epochs=3000, embedding_size=32, file_name="encoder_emb32_cora.pt")

In [ ]:
make_enc_embeds(data.x, num_features, num_epochs=3000, embedding_size=64, file_name="encoder_emb64_cora.pt")

Epoch [1/3000], Loss: 0.013141512870788574
Epoch [2/3000], Loss: 0.013021192513406277
Epoch [3/3000], Loss: 0.012914200313389301
Epoch [4/3000], Loss: 0.012818754650652409
Epoch [5/3000], Loss: 0.012733210809528828
Epoch [6/3000], Loss: 0.012656166218221188
Epoch [7/3000], Loss: 0.012586437165737152
Epoch [8/3000], Loss: 0.012523035518825054
Epoch [9/3000], Loss: 0.012465140782296658
Epoch [10/3000], Loss: 0.012412087060511112
Epoch [11/3000], Loss: 0.01236333604902029
Epoch [12/3000], Loss: 0.012318463064730167
Epoch [13/3000], Loss: 0.012277131900191307
Epoch [14/3000], Loss: 0.012239079922437668
Epoch [15/3000], Loss: 0.012204117141664028
Epoch [16/3000], Loss: 0.012172098271548748
Epoch [17/3000], Loss: 0.012142907828092575
Epoch [18/3000], Loss: 0.012116462923586369
Epoch [19/3000], Loss: 0.012092693708837032
Epoch [20/3000], Loss: 0.012071527540683746
Epoch [21/3000], Loss: 0.012052884325385094
Epoch [22/3000], Loss: 0.012036674655973911
Epoch [23/3000], Loss: 0.01202277932316064

In [ ]:
import statistics
from sklearn.metrics import f1_score

f1_score_list_base = []
acc_list_base = []

f1_score_list = []
acc_list = []
for i in range(10):
    print("*****")
    print(f'epoch : {i + 1} / 10')
    # model = GATv2(dataset.num_features, dataset.num_classes, embedding_dimension=32).to(device)
    model = GATv2(dataset.num_features, dataset.num_classes, embedding_dimension=8).to(device)
    # model = GCN(128, dataset.num_classes).to(device)
    acc_base, preds_base, labels_base = train_GNN_mode_and_save(model, data, 800, train_mask=new_train_mask,
                                                                test_mask=new_test_mask)
    preds_base = preds_base.cpu().numpy()
    labels_base = labels_base.cpu().numpy()
    f1_score_base = f1_score(labels_base, preds_base, average='macro')

    f1_score_list_base.append(f1_score_base)
    acc_list_base.append(acc_base)

    acc, preds, labels = finalizer_DNN(train_mask=new_train_mask, epochs=4000, test_mask=new_test_mask,
                                       repeat_threshold=0.90,
                                       file_name_enc_emb="encoder_emb32_cora.pt")

    preds = preds.cpu().numpy()
    labels = labels.cpu().numpy()

    print("base model")
    print(acc_base)
    print(f1_score_base)

    print("OurModel")
    print(acc)
    print(f1_score(labels, preds, average='macro'))
    f1_score_list.append(f1_score(labels, preds, average='macro'))
    acc_list.append(acc)

In [ ]:
print("base model")
print_details(acc_list_base, f1_score_list_base)
print("Our model")
print_details(acc_list, f1_score_list)

try 64

In [ ]:
GNN_model = GATv2(dataset.num_features, dataset.num_classes, embedding_dimension=8).to(device)
run_model(GNN_model , Gnn_epochs=900 ,enc_address="/content/encoder_emb64_cora.pt" ,reply_threshold=0.95 , head_epochs= 1400 , validation_mask=new_valdation_mask , early_stop=0.88 )

*****
epoch : 1 / 10
epoch : 50 = >  Accuracy: 0.3210
epoch : 100 = >  Accuracy: 0.4330
epoch : 150 = >  Accuracy: 0.4059
epoch : 200 = >  Accuracy: 0.3862
epoch : 250 = >  Accuracy: 0.3764
epoch : 300 = >  Accuracy: 0.3825
epoch : 350 = >  Accuracy: 0.4440
epoch : 400 = >  Accuracy: 0.5547
epoch : 450 = >  Accuracy: 0.6261
epoch : 500 = >  Accuracy: 0.7011
epoch : 550 = >  Accuracy: 0.7552
epoch : 600 = >  Accuracy: 0.8020
epoch : 650 = >  Accuracy: 0.8290
epoch : 700 = >  Accuracy: 0.8450
epoch : 750 = >  Accuracy: 0.8598
epoch : 800 = >  Accuracy: 0.8684
epoch : 850 = >  Accuracy: 0.8684
epoch : 900 = >  Accuracy: 0.8708
Final accuracy: 0.8708
epoch 0 ,Accuracy: 0.1697 , train acc : 0.17429837518463812
epoch 50 ,Accuracy: 0.7688 , train acc : 0.7754800590841949
epoch 100 ,Accuracy: 0.7847 , train acc : 0.792466765140325
epoch 150 ,Accuracy: 0.7798 , train acc : 0.8028064992614475
epoch 200 ,Accuracy: 0.7774 , train acc : 0.7976366322008862
epoch 250 ,Accuracy: 0.7823 , train acc : 0

In [ ]:
GNN_model = GAT(dataset.num_features, dataset.num_classes, embedding_dimension=8).to(device)
run_model(GNN_model , Gnn_epochs=700 ,enc_address="/content/encoder_emb64_cora.pt" ,reply_threshold=0.95 , head_epochs= 3000 , validation_mask=new_valdation_mask , early_stop=0.88 )

*****
epoch : 1 / 10
epoch : 50 = >  Accuracy: 0.3911
epoch : 100 = >  Accuracy: 0.3678
epoch : 150 = >  Accuracy: 0.3555
epoch : 200 = >  Accuracy: 0.3579
epoch : 250 = >  Accuracy: 0.3702
epoch : 300 = >  Accuracy: 0.4231
epoch : 350 = >  Accuracy: 0.5117
epoch : 400 = >  Accuracy: 0.5978
epoch : 450 = >  Accuracy: 0.6519
epoch : 500 = >  Accuracy: 0.7208
epoch : 550 = >  Accuracy: 0.7847
epoch : 600 = >  Accuracy: 0.8167
epoch : 650 = >  Accuracy: 0.8364
epoch : 700 = >  Accuracy: 0.8512
Final accuracy: 0.8512
epoch 0 ,Accuracy: 0.1255 , train acc : 0.15066469719350073
epoch 50 ,Accuracy: 0.3346 , train acc : 0.3190546528803545
epoch 100 ,Accuracy: 0.4194 , train acc : 0.39217134416543575
epoch 150 ,Accuracy: 0.4096 , train acc : 0.4224519940915805
epoch 200 ,Accuracy: 0.4736 , train acc : 0.5
epoch 250 ,Accuracy: 0.5006 , train acc : 0.5110782865583456
epoch 300 ,Accuracy: 0.5572 , train acc : 0.5745937961595273
epoch 350 ,Accuracy: 0.5633 , train acc : 0.5790251107828656
epoch 400

In [ ]:
GNN_model = GCN(64, dataset.num_classes).to(device)
run_model(GNN_model , Gnn_epochs=900 ,enc_address="/content/encoder_emb64_cora.pt" ,reply_threshold=0.95 , head_epochs= 1500 , validation_mask=new_valdation_mask , early_stop=0.89 )

*****
epoch : 1 / 10
epoch : 50 = >  Accuracy: 0.5375
epoch : 100 = >  Accuracy: 0.6162
epoch : 150 = >  Accuracy: 0.7146
epoch : 200 = >  Accuracy: 0.7663
epoch : 250 = >  Accuracy: 0.7970
epoch : 300 = >  Accuracy: 0.8093
epoch : 350 = >  Accuracy: 0.8216
epoch : 400 = >  Accuracy: 0.8549
epoch : 450 = >  Accuracy: 0.8672
epoch : 500 = >  Accuracy: 0.8819
epoch : 550 = >  Accuracy: 0.8856
epoch : 600 = >  Accuracy: 0.8881
epoch : 650 = >  Accuracy: 0.8918
epoch : 700 = >  Accuracy: 0.8918
epoch : 750 = >  Accuracy: 0.8881
epoch : 800 = >  Accuracy: 0.8881
epoch : 850 = >  Accuracy: 0.8881
epoch : 900 = >  Accuracy: 0.8868
Final accuracy: 0.8868
epoch 0 ,Accuracy: 0.4280 , train acc : 0.41432791728212703
epoch 50 ,Accuracy: 0.8303 , train acc : 0.8899556868537666
epoch 100 ,Accuracy: 0.8290 , train acc : 0.8943870014771049
epoch 150 ,Accuracy: 0.8278 , train acc : 0.9010339734121122
epoch 200 ,Accuracy: 0.8315 , train acc : 0.9076809453471196
epoch 250 ,Accuracy: 0.8315 , train acc : 